### Entrenamiento LSTM

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
data = pd.read_csv('df_merged_final.csv')
data


,Unnamed: 0,id,fecha_alta,id_municipio,id_estado,tipo_persona,genero,actividad_empresarial,fecha,comercio,giro_comercio,tipo_venta,edad,antiguedad_dias,monto
0,0,Usuario 1,2015-10-25,117,21,1,0,12,2022-01-02,25,72,0,40.0,2261,131.90
1,1,Usuario 1,2015-10-25,117,21,1,0,12,2022-01-02,6,22,0,40.0,2261,2.54
2,2,Usuario 1,2015-10-25,117,21,1,0,12,2022-01-02,6,22,0,40.0,2261,47.80
3,3,Usuario 1,2015-10-25,117,21,1,0,12,2022-01-02,59,74,1,40.0,2261,17.01
4,4,Usuario 1,2015-10-25,117,21,1,0,12,2022-01-04,59,74,1,40.0,2263,6.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346006,346006,Usuario 1000,2018-10-12,130,24,1,1,20,2023-01-27,44,4,0,36.0,1568,34.59
346007,346007,Usuario 1000,2018-10-12,130,24,1,1,20,2023-01-27,6,22,0,36.0,1568,190.83
346008,346008,Usuario 1000,2018-10-12,130,24,1,1,20,2023-01-27,77,22,0,36.0,1568,23.10
346009,346009,Usuario 1000,2018-10-12,130,24,1,1,20,2023-01-27,6,22,0,36.0,1568,56.52


In [3]:
data['fecha'] = pd.to_datetime(data['fecha'])

# Filtrar Usuario 1 y preparar datos
usuario_1 = data[data['id'] == 'Usuario 1'].copy()
usuario_1 = usuario_1.sort_values(by='fecha')
usuario_1.set_index('fecha', inplace=True)
serie = usuario_1['monto']

# Crear función para ventanas temporales
def crear_ventanas(serie, n_lags=5):
    df = pd.DataFrame()
    for i in range(n_lags):
        df[f'lag_{i+1}'] = serie.shift(i+1)
    df['target'] = serie.values
    df.dropna(inplace=True)
    return df

# Crear dataset supervisado
n_lags = 3
datos = crear_ventanas(serie, n_lags)

# Fechas límites para split
fecha_fin = datos.index.max()
fecha_validacion_inicio = fecha_fin - pd.DateOffset(months=1)
fecha_entrenamiento_inicio = fecha_fin - pd.DateOffset(months=5)

# Filtrar conjunto de entrenamiento y validación
datos_filtrados = datos[(datos.index >= fecha_entrenamiento_inicio)]

entrenamiento = datos_filtrados[datos_filtrados.index < fecha_validacion_inicio]
validacion = datos_filtrados[datos_filtrados.index >= fecha_validacion_inicio]

# Separar variables
X_train, y_train = entrenamiento.drop('target', axis=1).values, entrenamiento['target'].values
X_test, y_test = validacion.drop('target', axis=1).values, validacion['target'].values


### LSTM

In [4]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Parámetro de ventanas
timesteps = 5

# Función para crear secuencias
def crear_secuencias(X, y, timesteps):
    X_seqs, y_seqs = [], []
    for i in range(len(X) - timesteps):
        X_seqs.append(X[i:i+timesteps])
        y_seqs.append(y[i+timesteps])
    return np.array(X_seqs), np.array(y_seqs)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear secuencias
X_train_seq, y_train_seq = crear_secuencias(X_train_scaled, y_train, timesteps)
X_test_seq, y_test_seq = crear_secuencias(X_test_scaled, y_test, timesteps)

# Asegurar dimensiones para LSTM
if X_train_seq.ndim == 2:
    X_train_seq = np.expand_dims(X_train_seq, -1)
    X_test_seq = np.expand_dims(X_test_seq, -1)

# Arquitectura a usar 
estructura = (64,) 

# Semilla para reproducibilidad
semilla = 42
tf.keras.backend.clear_session()
tf.random.set_seed(semilla)
np.random.seed(semilla)

# Crear el modelo
model = Sequential()
model.add(LSTM(estructura[0], activation='tanh', input_shape=(timesteps, X_train_seq.shape[2])))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.01), loss=MeanSquaredError())

# Entrenamiento
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train_seq, y_train_seq,
          validation_split=0.1,
          epochs=200,
          batch_size=16,
          verbose=0,
          callbacks=[early_stop])

# Predicción y cálculo del RMSE
y_pred = model.predict(X_test_seq).flatten()
mse_metric = MeanSquaredError()

# Calcular el MSE
mse_value = mse_metric(y_test_seq, y_pred).numpy()  # Devuelve un tensor, convertimos a número

# Calcular RMSE
rmse = np.sqrt(mse_value)

print(f'RMSE: {rmse:.4f}')


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
RMSE: 16.8864


In [5]:
# Número de pasos hacia adelante que quieres predecir
pasos_a_predecir = 10

# Tomamos la última secuencia real del conjunto de validación para iniciar las predicciones
ultima_secuencia_real = X_test_seq[-1]  # Forma: (timesteps, num_features)

# Para almacenar las predicciones futuras
predicciones_futuras = []

# Copiamos para ir actualizando la secuencia
secuencia_actual = ultima_secuencia_real.copy()

for _ in range(pasos_a_predecir):
    # Predecir el siguiente valor
    prediccion = model.predict(secuencia_actual[np.newaxis, :, :])[0,0]
    
    # Guardar la predicción
    predicciones_futuras.append(prediccion)
    
    # Crear la nueva secuencia para la siguiente predicción:
    #  - eliminamos el primer paso de tiempo
    #  - añadimos la predicción al final
    # Nota: Debemos saber qué features se usan (lags)
    
    # Para tu caso, tienes lags como features, vamos a asumir que sólo hay 1 feature (monto escalado)
    # Si hay más features, ajustar
    
    # Construimos el nuevo input desplazado
    nueva_secuencia = np.roll(secuencia_actual, shift=-1, axis=0)
    # Colocamos la nueva predicción en el último lugar
    nueva_secuencia[-1, 0] = prediccion  # si solo 1 feature
    
    # Actualizamos secuencia actual
    secuencia_actual = nueva_secuencia

# Las predicciones están en escala estándar, si quieres el valor original hay que desescalar
# Como solo escalaste X (lags), pero no y, puede que tengas que revertir escalado manualmente
# Si el target estaba en escala original, entonces ya están en esa escala

print("Predicciones hacia adelante:")
print(predicciones_futuras)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Predicciones hacia adelante:
[27.45499, 27.283092, 27.171764, 25.862907, 15.013011, 16.16529, 19.42158, 20.79771, 18.519577, 16.915648]


In [6]:
def predecir_usuario_adelante(data, usuario_id, modelo, scaler, n_lags, timesteps, pasos_a_predecir):
    """
    Realiza predicciones hacia adelante para un usuario específico con un modelo ya entrenado.
    
    Parámetros:
    - data: DataFrame completo con columnas 'id', 'fecha', 'monto'.
    - usuario_id: id del usuario a predecir (string).
    - modelo: modelo Keras LSTM ya entrenado.
    - scaler: StandardScaler ya ajustado con los datos de entrenamiento.
    - n_lags: número de lags usados para crear el dataset supervisado.
    - timesteps: número de timesteps para las secuencias LSTM.
    - pasos_a_predecir: número de pasos (días) a predecir hacia adelante.
    
    Retorna:
    - lista con las predicciones hacia adelante (escaladas).
    """
    # 1. Filtrar y preparar serie para el usuario
    usuario = data[data['id'] == usuario_id].copy()
    usuario = usuario.sort_values(by='fecha')
    usuario.set_index('fecha', inplace=True)
    serie = usuario['monto']

    # 2. Crear dataset supervisado
    datos_usuario = crear_ventanas(serie, n_lags)

    # 3. Definir fechas para validar
    fecha_fin = datos_usuario.index.max()
    fecha_validacion_inicio = fecha_fin - pd.DateOffset(months=1)
    fecha_entrenamiento_inicio = fecha_fin - pd.DateOffset(months=5)

    datos_filtrados = datos_usuario[(datos_usuario.index >= fecha_entrenamiento_inicio)]
    validacion = datos_filtrados[datos_filtrados.index >= fecha_validacion_inicio]

    X_test, y_test = validacion.drop('target', axis=1).values, validacion['target'].values

    # 4. Escalar (usar scaler ya ajustado)
    X_test_scaled = scaler.transform(X_test)

    # 5. Crear secuencias
    X_test_seq, y_test_seq = crear_secuencias(X_test_scaled, y_test, timesteps)
    if X_test_seq.ndim == 2:
        X_test_seq = np.expand_dims(X_test_seq, -1)

    # 6. Predicciones hacia adelante
    ultima_secuencia_real = X_test_seq[-1].copy()
    predicciones_futuras = []
    secuencia_actual = ultima_secuencia_real.copy()

    for _ in range(pasos_a_predecir):
        prediccion = modelo.predict(secuencia_actual[np.newaxis, :, :])[0, 0]
        predicciones_futuras.append(prediccion)
        nueva_secuencia = np.roll(secuencia_actual, shift=-1, axis=0)
        nueva_secuencia[-1, 0] = prediccion
        secuencia_actual = nueva_secuencia

    return predicciones_futuras


In [7]:
preds_usuario3 = predecir_usuario_adelante(
    data=data,
    usuario_id='Usuario 9',
    modelo=model,
    scaler=scaler,
    n_lags=3,
    timesteps=5,
    pasos_a_predecir=10
)

print(preds_usuario3)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
[27.463509, 27.266935, 27.24439, 19.844118, 15.226793, 17.326189, 20.27627, 20.259523, 17.372612, 17.23321]


## Guardar

In [8]:
import joblib
from tensorflow.keras.models import load_model

# --- Guardar scaler ---
joblib.dump(scaler, 'scaler_model.save')

# --- Guardar modelo ---
model.save('modelo_lstm.h5')

# --- Cargar scaler ---
scaler_cargado = joblib.load('scaler_model.save')

# --- Cargar modelo ---
model_cargado = load_model('modelo_lstm.h5')



In [ ]:
preds_usuario3 = predecir_usuario_adelante(
    data=data,
    usuario_id='Usuario 3',
    modelo=model_cargado,
    scaler=scaler_cargado,
    n_lags=3,
    timesteps=5,
    pasos_a_predecir=10 # días
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
